In [12]:
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn import cross_validation

In [13]:
FEATURES = [
    "gd",
    "ed",
    "gp",
    "isr",
#    "views", 
    "large", 
     "top", 
#    "speed_level",
#    "speed_level 1",
#    "speed_level 2",
#    "speed_level 3",
    "price"]

In [14]:
def get_speed_level(speed_level):
    if speed_level == 1:
        return  [1]
        #return [1,0,0]
        #print X[-1], speed_level
    elif speed_level == 2:
        return [0.4]
        #return [0,1,0]
        #print X[-1], speed_level
    elif speed_level == 3:
        return [0.1]
        #return [0,0,1]
        #print X[-1], speed_level
    else:
        return [0]
        #return [0,0,0]
        #print X[-1], speed_level

In [15]:
def extract_data(clinic, clinic_estim):
    all_proc = clinic['procedures']['diagnostics'] + clinic['procedures']['operations']
    all_doc = clinic['doctors']
    X = []
    for doc in all_doc:
        X+=[[]]
        #gd 0-7
        X[-1]+=[doc['degree'].count(u'Профессор')]
        X[-1]+=[int(doc['top'])]
        X[-1]+=[len(doc['illnesses'])]
        X[-1]+=[len(doc['languages'])]
        X[-1]+=[int(doc['h_index'])]
        X[-1]+=[int(doc['experience'])]
        X[-1]+=[1]
        #ed 7-12
        ids_diad = [i['id'] for i in doc['procedures']['diagnostics'] if i['top']=='1']
        ids_oper = [i['id'] for i in doc['procedures']['operations'] if i['top']=='1']
        X[-1]+=[len(list(set(ids_diad)))]
        X[-1]+=[len(doc['procedures']['diagnostics'])]
        X[-1]+=[len(list(set(ids_oper)))]
        X[-1]+=[len(doc['procedures']['operations'])]
        X[-1]+=[1]
        #gp 12-16
        X[-1]+=[sum([int(i['top']) for i in all_proc])] #top_proc_cli
        X[-1]+=[len(all_proc)] #gen_proc_cli
        X[-1]+=[sum([int(i['top']) for i in clinic['procedures']['operations']])] #top_oper_cli
        X[-1]+=[len(clinic['procedures']['operations'])] #gen_oper_cli
        #
        c_id = int(clinic['country_id'])
        if c_id == 1:
            X[-1]+=[1]
        else:
            X[-1]+=[0]
        #X[-1]+=[int(clinic['views'])]
        X[-1]+=[int(clinic['large'])]
        X[-1]+=[int(clinic['top'])]
        #X[-1]+=[get_speed_level(int(clinic['speed_level']))]
        X[-1]+=[round(1-int(clinic_estim['price'])/10.0,4)]
        
    y=[int(clinic_estim['rating'])] * len(all_doc)
    return [X, y]

In [16]:
def get_X_y_from(t_data):
    clinic_ids = [i['id'] for i in t_data]
    t_cleaned_data = [extract_data(cancer_data[clinic_ids.index(i['id'])], i) for i in estims_data]
    X = sum([i[0] for i in t_cleaned_data],[])
    y = sum([i[1] for i in t_cleaned_data],[])
    return X,y

In [17]:
def normale(mat):
    for i in range(0,len(mat[0])-1):
        mass = [abs(k[i]) for k in mat]
        m = max(mass)
        if m>0:
            for j in mat:
                j[i] = round(j[i] / float(m), 4)

In [18]:
def pack(X):
    gd = [0.06, 0.223, 0.372, 0.196, -0.135, 0.426, 8.37]
    ed = [-2.82, -1.104, 0.827, 0.96, 9.471]
    gp = [-3.094, -7.545, 11.025, 9.011]
    new_X = []
    for x in X:
        new_X+= [sum([
            [round(sum(np.array(gd)*np.array(x[0:7])),5)],
            [round(sum(np.array(ed)*np.array(x[7:12])),5)],
            [round(sum(np.array(gp)*np.array(x[12:16])),5)],
            x[16:]
        ],[])]
    return new_X

In [19]:
def process_with(X,y, info=False, short=False, return_short = False, new_coef = [], ts=0.2):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = ts, random_state = 3)
    regr = linear_model.LinearRegression(fit_intercept=True, normalize=True)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    return regr

In [20]:
X, y = get_X_y_from(cancer_data)

NameError: name 'cancer_data' is not defined

In [21]:
X_all, y_all = [], []

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[2]['clinics']
with open('./max/меланома_все.json') as data_file: #2
    cancer_data = json.load(data_file)[0]['clinics']
X_1, y_1 = get_X_y_from(cancer_data)
X_all+=X_1
y_all+=y_1
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[0]['clinics']
with open('./max/рак_груди_все.json') as data_file: #0
    cancer_data = json.load(data_file)[0]['clinics']
X_2, y_2 = get_X_y_from(cancer_data)
X_all+=X_2
y_all+=y_2
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[4]['clinics']    
with open('./max/рак_простаты_все.json') as data_file: #4
    cancer_data = json.load(data_file)[0]['clinics']
clinic_ids = [i['id'] for i in estims_data]
estims_data.pop(clinic_ids.index('0'))
X_3, y_3 = get_X_y_from(cancer_data)
X_all+=X_3
y_all+=y_3

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[1]['clinics']     
with open('./max/рак_шейки_матки_все.json') as data_file: #1
    cancer_data = json.load(data_file)[0]['clinics']
X_4, y_4 = get_X_y_from(cancer_data)
X_all+=X_4
y_all+=y_4
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[3]['clinics']         
with open('./max/рак_щитовидки_все.json') as data_file: #3
    cancer_data = json.load(data_file)[0]['clinics']
X_5, y_5 = get_X_y_from(cancer_data)
X_all+=X_5
y_all+=y_5

normale(X_all)
X_all=pack(X_all)
normale(X_all)

In [26]:
import pandas as pd
data_x = pd.DataFrame.from_records(X_all, columns=FEATURES)
data_y = pd.DataFrame()
data_y["rate"] = y_all

top_count = sum(data_x["top"])
large_count = sum(data_x["large"])
print top_count
print large_count
data = data_x
data["y"] = data_y["rate"]



35.0
51.0


In [61]:
print data.columns

data_large = data[data['large'] == 1]

data_not_large = data[data['large'] == 0]


import matplotlib.pyplot as plt
plt.plot(data_large['price'], data_large['y'], 'ro', data_not_large['price'], data_not_large['y'], 'bo')

plt.show()


Index([u'gd', u'ed', u'gp', u'isr', u'large', u'top', u'price', u'y', u'top_or_large'], dtype='object')


In [29]:

data_large = data[data['large'] == 1]
data_large = data_large.groupby(['y'])['large'].sum()

data_not_large = data[data['large'] == 0]
data_not_large["large"] = data_not_large["large"].map(lambda x: 1)
data_not_large = data_not_large.groupby(['y'])['large'].sum()
print "Large:"
print data_large

print "Not large:"
print data_not_large

data_top = data[data['top'] == 1]
data_top = data_top.groupby(['y'])['top'].sum()

data_not_top = data[data['top'] == 0]
data_not_top["top"] = data_not_top["top"].map(lambda x: 1)
data_not_top = data_not_top.groupby(['y'])['top'].sum()
print "Top:"
print data_top

print "Not top:"
print data_not_top

data_isr = data[data['isr'] == 1]
data_isr = data_isr.groupby(['y'])['isr'].sum()

data_not_isr = data[data['isr'] == 0]
data_not_isr["isr"] = data_not_isr["isr"].map(lambda x: 1)
data_not_isr = data_not_isr.groupby(['y'])['isr'].sum()
print "Israel:"
print data_isr

print "Not Israel:"
print data_not_isr

Large:
y
1      2
2      3
3      1
4      2
5     11
6      8
7      3
8      4
9      4
10    13
Name: large, dtype: float64
Not large:
y
1    2
2    2
3    1
4    2
7    1
8    1
9    4
Name: large, dtype: int64
Top:
y
2      1
5     10
6      7
9      4
10    13
Name: top, dtype: float64
Not top:
y
1    4
2    4
3    2
4    4
5    1
6    1
7    4
8    5
9    4
Name: top, dtype: int64
Israel:
y
2      1
5     10
6      7
9      4
10    13
Name: isr, dtype: float64
Not Israel:
y
1    4
2    4
3    2
4    4
5    1
6    1
7    4
8    5
9    4
Name: isr, dtype: int64


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


In [35]:
import matplotlib.pyplot as plt

plt.figure(1)
plt.subplot(211)
top, = plt.plot(data_top.index, data_top.values, label='Top')
not_top, = plt.plot(data_not_top.index, data_not_top.values, label='Not top')
plt.legend(handles=[top, not_top])


plt.subplot(212)
large, = plt.plot(data_large.index, data_large.values, label='Large')
not_large, = plt.plot(data_not_large.index, data_not_large.values, label='Not large')
plt.legend(handles=[large, not_large])
plt.show()

plt.figure(2) 
isr, = plt.plot(data_isr.index, data_isr.values, label='Israel')
not_isr, = plt.plot(data_not_isr.index, data_not_isr.values, label='Not Israel')
plt.legend(handles=[isr, not_isr])

plt.show()



(64, 8)


In [12]:
len(X_all[1])

/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormat

7

In [13]:
len(cancer_data[3]['doctors'])

1

In [14]:
X_all

[[0.9756, 0.9695, 1.0, 1.0, 1.0, 1.0, 0.5],
 [0.9269, 0.9782, 1.0, 1.0, 1.0, 1.0, 0.5],
 [1.0, 0.8955, 1.0, 1.0, 1.0, 1.0, 0.5],
 [0.9467, 0.9673, 1.0, 1.0, 1.0, 1.0, 0.5],
 [0.9225, 0.926, 1.0, 1.0, 1.0, 1.0, 0.5],
 [0.9164, 0.9827, 1.0, 1.0, 1.0, 1.0, 0.5],
 [0.895, 0.8628, 0.289, 0.0, 0.0, 0.0, 0.4],
 [0.8976, 0.9129, 0.1516, 0.0, 1.0, 0.0, 0.3],
 [0.9465, 1.0, 0.9469, 1.0, 1.0, 1.0, 0.7],
 [0.9313, 0.9129, 0.9469, 1.0, 1.0, 1.0, 0.7],
 [0.8798, 0.8606, -0.2452, 0.0, 0.0, 0.0, 0.6],
 [0.8998, 0.8126, 0.4241, 0.0, 1.0, 0.0, 0.3],
 [0.9042, 0.8737, -0.14, 0.0, 0.0, 0.0, 0.1],
 [0.9465, 1.0, 0.9469, 1.0, 1.0, 1.0, 0.6],
 [0.9354, 0.9673, 0.9469, 1.0, 1.0, 1.0, 0.6],
 [0.9244, 0.9891, 0.9469, 1.0, 1.0, 1.0, 0.6],
 [0.9184, 0.9695, 0.9469, 1.0, 1.0, 1.0, 0.6],
 [0.9261, 0.8715, 0.1516, 0.0, 1.0, 0.0, 0.3],
 [0.8979, 0.9129, 0.1516, 0.0, 1.0, 0.0, 0.3],
 [0.9031, 0.8126, 0.1516, 0.0, 1.0, 0.0, 0.3],
 [0.9756, 0.9695, 1.0, 1.0, 1.0, 1.0, 0.4],
 [0.9269, 0.9782, 1.0, 1.0, 1.0, 1.0, 0.4],
 [

In [62]:
for i in range(1,10):
    print i
    regr = process_with(X_all, y_all, info=True, ts=i/10.0)
    print

1
Total: 64, train: 57, test: 7
Residual sum of squares: 10.50
Train absolute: 2.14
Test absolute: 2.99
Absolute to mean: 46.53%
Train variance score: 0.29
Test variance score: -0.61

2
Total: 64, train: 51, test: 13
Residual sum of squares: 8.25
Train absolute: 2.19
Test absolute: 2.59
Absolute to mean: 43.16%
Train variance score: 0.24
Test variance score: 0.06

3
Total: 64, train: 44, test: 20
Residual sum of squares: 8.29
Train absolute: 2.23
Test absolute: 2.52
Absolute to mean: 40.62%
Train variance score: 0.27
Test variance score: -0.11

4
Total: 64, train: 38, test: 26
Residual sum of squares: 7.19
Train absolute: 2.34
Test absolute: 2.44
Absolute to mean: 38.40%
Train variance score: 0.28
Test variance score: -0.08

5
Total: 64, train: 32, test: 32
Residual sum of squares: 7.54
Train absolute: 2.24
Test absolute: 2.29
Absolute to mean: 37.33%
Train variance score: 0.34
Test variance score: -0.10

6
Total: 64, train: 25, test: 39
Residual sum of squares: 7.30
Train absolute: 2.

In [79]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn import linear_model

clf = RandomForestClassifier(n_estimators=50)
#clf = linear_model.LinearRegression()

#X_train, X_test, y_train, y_test = train_test_split(X_all, Y_all, test_size=0.33, random_state=42)
#X = X_all[1:3]
for row in X_all:
    del row[3]
    del row[5]


clf = clf.fit(X_all, y_all)

"""
print '\n\nSplitting data to training and test sets:'
print ('Mean square error: %.2f' % np.mean((clf.predict(X_test) - y_test)**2))
print ('Accuracy score: %f' % accuracy_score(y_test, gbm.predict(X_test)))
print ('Precision score: %f' % precision_score(y_test, gbm.predict(X_test),average='binary'))
print ('Recall score: %f' % recall_score(y_test, gbm.predict(X_test),average='binary'))
"""
importances = clf.feature_importances_
print importances
print FEATURES
#print X_all



[ 0.33150995  0.32088703  0.24251824  0.03595154  0.06913325]
['gd', 'ed', 'gp', 'isr', 'large', 'top', 'price']


In [135]:
regr = process_with(X_all, y_all, info=True, ts=0.6)

Total: 64, train: 25, test: 39
Residual sum of squares: 7.01
Train absolute: 2.37
Test absolute: 2.21
Absolute to mean: 34.41%
Train variance score: 0.27
Test variance score: 0.07


In [136]:
import warnings
warnings.filterwarnings("ignore")
for i in range(0,len(y_all)):    
    print "Real: %f \t Predicted: %f" %(y_all[i], regr.predict(X_all[i]))
    print sum(X_all[i])
    print
    #print y_all[i]
    #print regr.predict(X_all[i])[0]

Real: 10.000000 	 Predicted: 7.704660
5.9451

Real: 10.000000 	 Predicted: 7.070325
5.9051

Real: 10.000000 	 Predicted: 7.309293
5.8955

Real: 10.000000 	 Predicted: 7.252825
5.914

Real: 10.000000 	 Predicted: 6.470389
5.8485

Real: 10.000000 	 Predicted: 6.960433
5.8991

Real: 9.000000 	 Predicted: 5.657148
2.0468

Real: 8.000000 	 Predicted: 5.510347
2.9621

Real: 6.000000 	 Predicted: 7.315954
5.8934

Real: 6.000000 	 Predicted: 6.198178
5.7911

Real: 4.000000 	 Predicted: 2.704850
1.4952

Real: 3.000000 	 Predicted: 5.895186
3.1365

Real: 1.000000 	 Predicted: 3.733958
1.6379

Real: 9.000000 	 Predicted: 7.315954
5.8934

Real: 9.000000 	 Predicted: 6.816188
5.8496

Real: 9.000000 	 Predicted: 6.876036
5.8604

Real: 9.000000 	 Predicted: 6.586185
5.8348

Real: 7.000000 	 Predicted: 5.509737
2.9492

Real: 7.000000 	 Predicted: 5.514804
2.9624

Real: 7.000000 	 Predicted: 4.564887
2.8673

Real: 5.000000 	 Predicted: 7.704660
5.9451

Real: 5.000000 	 Predicted: 7.070325
5.9051

Real:

In [117]:
print "Features sorted by their score:"
#f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
print regr.intercept_
print len(FEATURES)
for i in f:
    print "%4f \t %s" %(i[0], i[1])
    #print i[0] #, i[1]

Features sorted by their score:
-9.60471591456
5
10.809400 	 gd
2.475900 	 ed
1.121000 	 gp
0.476800 	 large
5.097000 	 price


In [ ]:
ind = [map(lambda x: x[1], f).index(i) for i in FEATURES]
for i in ind:
    print i+1

In [61]:
plt.plot(sorted([i[2] for i in X_all]))
plt.show()

In [ ]:
print FEATURES[8]
[i[22] for i in X]

In [ ]:
for k in range(0,26):
    print FEATURES[k], 10 - [i[k] for i in X].count(0)